In [9]:
# Combined driving: 30.01_quaternary steering + traffic-light stop + velocity control (PID)
# - Uses your steering classifier to self-steer
# - Predicts traffic lights and applies a HARD STOP on red (with debounced release)
# - Predicts target speed from a velocity regressor and controls throttle/brake via PID
# - Preserves your HUD and left-turn degree tracker from trafficModelDriving
#
# Key model files expected in ../Models/ :
#   - 30.01_quaternary_model_final.pth              (steering classifier)
#   - traffic_classifier_state.pth (+ class_mapping.json optional)  (traffic-light classifier)
#   - velocity_regressor_split.pth                  (velocity regressor)
#
# Controls:
#   Q/E/R  : Left / Right / Cancel turn signal
#   P      : Respawn vehicle
#   ESC    : Quit
#
# Notes:
#   • If traffic-light weights are missing, TL overlay shows N/A and stops are disabled.
#   • Velocity regressor defines the target speed (m/s). PID tracks it unless HARD STOP is active.

import os
import json
import time
import random
from pathlib import Path

import numpy as np
import pygame
import carla

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models, transforms
from PIL import Image
from carla import VehicleLightState
import cv2  # for resizing / HUD effects

# ========= Steering model (YOUR 30.01_quaternary) =========
CLASS_NAMES_STEER = [
    "Left Hardest", "Left Harder", "Left Hard", "Left Medium", "Left Light", "Left Slight", "Left Minimal",
    "No Turning",
    "Right Minimal", "Right Slight", "Right Light", "Right Medium", "Right Hard", "Right Harder", "Right Hardest"
]

class SteeringClassifier(nn.Module):
    def __init__(self, num_classes=15):
        super().__init__()
        try:
            resnet = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
        except Exception:
            resnet = models.resnet18(pretrained=True)
        resnet.fc = nn.Identity()
        self.cnn = resnet
        self.turn_embed = nn.Embedding(3, 16)
        self.fc = nn.Sequential(
            nn.Linear(512 + 16, 128),
            nn.ReLU(),
            nn.Linear(128, num_classes)
        )

    def forward(self, image, turn_signal):
        x_img = self.cnn(image)
        x_signal = self.turn_embed(turn_signal)
        x = torch.cat((x_img, x_signal), dim=1)
        return self.fc(x)

# ========= Velocity regression model =========
class VelocityRegression(nn.Module):
    def __init__(self):
        super().__init__()
        try:
            base = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
        except Exception:
            base = models.resnet18(pretrained=True)
        base.fc = nn.Linear(base.fc.in_features, 1)
        self.model = base

    def forward(self, image):
        return self.model(image)

# ========= Traffic-light classifier =========
SAVE_DIR       = "../Models"
WEIGHTS_PTH_TL = os.path.join(SAVE_DIR, "traffic_classifier_state.pth")
CLASS_MAP_JSON = os.path.join(SAVE_DIR, "class_mapping.json")

label_to_index = {-1: 0, 0: 1, 1: 2}
index_to_label = {v: k for k, v in label_to_index.items()}
CLASS_NAMES_TL = {-1: "No Light", 0: "Red", 1: "Green"}
CLASS_EMOJI_TL = {-1: "⚫️", 0: "🔴", 1: "🟢"}

if os.path.exists(CLASS_MAP_JSON):
    with open(CLASS_MAP_JSON, "r") as f:
        m = json.load(f)
    if "label_to_index" in m and "index_to_label" in m:
        label_to_index = {int(k): int(v) for k, v in m["label_to_index"].items()}
        index_to_label = {int(k): int(v) for k, v in m["index_to_label"].items()}

NUM_CLASSES_TL = 3
IN_CHANNELS_TL = 3  # camera is RGB

class TLResNetClassifier(nn.Module):
    def __init__(self, in_channels=3, num_classes=3):
        super().__init__()
        try:
            from torchvision.models import resnet18
            backbone = resnet18(weights=None)
        except Exception:
            from torchvision.models import resnet18
            backbone = resnet18(pretrained=False)

        if in_channels != 3:
            backbone.conv1 = nn.Conv2d(in_channels, 64, kernel_size=7, stride=2, padding=3, bias=False)
        in_feat = backbone.fc.in_features
        backbone.fc = nn.Sequential(nn.Dropout(0.2), nn.Linear(in_feat, num_classes))
        self.net = backbone

    def forward(self, x):
        if x.dim() == 4:
            _, _, H, W = x.shape
            if H < 224 or W < 224:
                x = F.interpolate(x, size=(224, 224), mode="bilinear", align_corners=False)
        return self.net(x)

# ========= Initialize models =========
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

steer_model = SteeringClassifier(num_classes=15).to(device)
steer_model.load_state_dict(torch.load(os.path.join(SAVE_DIR, "30.01_quaternary_model_final.pth"), map_location=device))
steer_model.eval()

velocity_model = VelocityRegression().to(device)
velocity_model.load_state_dict(torch.load(os.path.join(SAVE_DIR, "velocity_regressor_split.pth"), map_location=device))
velocity_model.eval()

tl_model = TLResNetClassifier(IN_CHANNELS_TL, NUM_CLASSES_TL).to(device)
if not os.path.exists(WEIGHTS_PTH_TL):
    print(f"[WARN] Traffic-light weights not found at {WEIGHTS_PTH_TL}. Overlay will show N/A and red-stop disabled.")
    TL_AVAILABLE = False
else:
    state = torch.load(WEIGHTS_PTH_TL, map_location=device)
    tl_state_dict = state["model_state_dict"] if isinstance(state, dict) and "model_state_dict" in state else state
    tl_model.load_state_dict(tl_state_dict, strict=True)
    TL_AVAILABLE = True

tl_model.eval()

# ========= Transforms =========
image_transform_steervel = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

IMAGENET_MEAN = (0.485, 0.456, 0.406)
IMAGENET_STD  = (0.229, 0.224, 0.225)

@torch.no_grad()
def predict_traffic_light(np_rgb):
    if not TL_AVAILABLE:
        return "N/A", 0.0

    x = np.asarray(np_rgb)
    if x.ndim == 2:
        x = x[..., None]
    if x.dtype != np.float32:
        x = x.astype(np.float32)
    if x.max() > 1.5:
        x /= 255.0

    x = np.transpose(x, (2, 0, 1))

    if x.shape[0] != IN_CHANNELS_TL:
        if IN_CHANNELS_TL == 3 and x.shape[0] == 1:
            x = np.repeat(x, 3, axis=0)
        elif IN_CHANNELS_TL == 1 and x.shape[0] == 3:
            x = x.mean(axis=0, keepdims=True)
        else:
            raise ValueError(f"Channel mismatch for TL model: got {x.shape[0]}, expected {IN_CHANNELS_TL}")

    xt = torch.from_numpy(x).unsqueeze(0).to(device)
    _, _, H, W = xt.shape
    if H != 224 or W != 224:
        xt = F.interpolate(xt, size=(224, 224), mode="bilinear", align_corners=False)

    if IN_CHANNELS_TL == 3:
        mean = torch.tensor(IMAGENET_MEAN, device=device).view(1, 3, 1, 1)
        std  = torch.tensor(IMAGENET_STD,  device=device).view(1, 3, 1, 1)
        xt = (xt - mean) / std
    else:
        xt = (xt - 0.5) / 0.5

    logits = tl_model(xt)
    probs = torch.softmax(logits, dim=1).squeeze(0)
    conf, pred_idx = torch.max(probs, dim=0)

    pred_label = index_to_label[int(pred_idx.item())]
    name  = CLASS_NAMES_TL.get(pred_label, str(pred_label))
    emoji = CLASS_EMOJI_TL.get(pred_label, "")
    return f"{emoji} {name}", float(conf.item())

# ========= pygame + CARLA =========
pygame.init()
width, height = 1024, 768
screen = pygame.display.set_mode((width, height))
pygame.display.set_caption("CARLA: Steering + TL + Velocity PID")
font = pygame.font.SysFont(None, 30)
font_big = pygame.font.SysFont(None, 36)

client = carla.Client("localhost", 2000)
client.set_timeout(25.0)
client.load_world("Town05")
world = client.get_world()
bp = world.get_blueprint_library()

vehicle_bp = (bp.filter("vehicle.dodge.charger_2020") or bp.filter("vehicle.*model3*"))[0]
spawn_point = random.choice(world.get_map().get_spawn_points())
vehicle = world.spawn_actor(vehicle_bp, spawn_point)
vehicle.set_autopilot(False)

spectator = world.get_spectator()

# Front narrow camera (steer/vel)
cam_bp = bp.find('sensor.camera.rgb')
cam_bp.set_attribute('image_size_x', '448')
cam_bp.set_attribute('image_size_y', '252')
cam_bp.set_attribute('fov', '145')
cam_bp.set_attribute('sensor_tick', '0.1')
cam_transform = carla.Transform(carla.Location(x=1.5, z=2.4))
camera = world.spawn_actor(cam_bp, cam_transform, attach_to=vehicle)

# Wide HUD / TL camera
cam_bp2 = bp.find('sensor.camera.rgb')
cam_bp2.set_attribute('image_size_x', '1024')
cam_bp2.set_attribute('image_size_y', '768')
cam_bp2.set_attribute('fov', '110')
cam_bp2.set_attribute('sensor_tick', '0.1')
cam_transform2 = carla.Transform(carla.Location(x=1.5, z=2.4))
camera2 = world.spawn_actor(cam_bp2, cam_transform2, attach_to=vehicle)

camera_image = None
camera_np = None
camera_image2 = None
camera_np2 = None
pred_label_steer = "Loading..."
pred_label_tl = "N/A"
conf_tl = 0.0

# ========= Helper image processing =========
def traffic_light_crop(img: np.ndarray) -> np.ndarray:
    if img is None or img.ndim != 3:
        return img

    h, w = img.shape[:2]
    out = img

    # Center zoom 50%
    z = 1.0 + (50.0 / 100.0)
    new_w = max(1, int(w / z))
    new_h = max(1, int(h / z))
    x1 = max(0, (w - new_w) // 2)
    y1 = max(0, (h - new_h) // 2)
    cropped = out[y1:y1 + new_h, x1:x1 + new_w]
    if cropped.size > 0:
        out = cv2.resize(cropped, (w, h))

    # Blackout bottom third
    out[h * 2 // 3 :, ...] = 0

    # Blur+desaturate left/right edges for focus (keep color overall for UI)
    quarter_w = max(1, w // 4)
    def blur_and_desaturate(region: np.ndarray) -> np.ndarray:
        blurred = cv2.GaussianBlur(region, (21, 21), 0)
        hsv = cv2.cvtColor(blurred, cv2.COLOR_BGR2HSV)
        hsv[:, :, 1] = 0
        desat = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
        return desat

    out[:, :quarter_w, :] = blur_and_desaturate(out[:, :quarter_w, :])
    out[:, -quarter_w:, :] = blur_and_desaturate(out[:, -quarter_w:, :])

    return out

# ========= Camera listeners =========
def process_image(image):
    global camera_image, camera_np
    array = np.frombuffer(image.raw_data, dtype=np.uint8)
    array = array.reshape((image.height, image.width, 4))
    array = array[:, :, :3][:, :, ::-1]  # RGB
    camera_np = array.copy()
    camera_image_raw = pygame.surfarray.make_surface(array.swapaxes(0, 1))
    camera_image = pygame.transform.scale(camera_image_raw, (width, height))

def process_image2(image):
    global camera_image2, camera_np2, left_signal_on
    array = np.frombuffer(image.raw_data, dtype=np.uint8)
    array = array.reshape((image.height, image.width, 4))
    array = array[:, :, :3][:, :, ::-1]  # BGRA -> RGB

    array = traffic_light_crop(array)

    # Right-anchored grayscale rectangle (width ~ 1/3, height ~ 1/2)
    H, W = array.shape[:2]
    rect_w = (2 * W) // 6
    rect_h = H // 4

    # top right
    y0 = 0; y1 = rect_h * 2
    x1 = W; x0 = max(0, x1 - rect_w)
    roi_r = array[y0:y1, x0:x1]
    gray_r = cv2.cvtColor(roi_r, cv2.COLOR_RGB2GRAY)
    array[y0:y1, x0:x1] = cv2.cvtColor(gray_r, cv2.COLOR_GRAY2RGB)

    # top left
    y0 = 0; y1 = rect_h
    x1 = max(0, rect_w); x0 = 0
    roi_l = array[y0:y1, x0:x1]
    gray_l = cv2.cvtColor(roi_l, cv2.COLOR_RGB2GRAY)
    array[y0:y1, x0:x1] = cv2.cvtColor(gray_l, cv2.COLOR_GRAY2RGB)

    camera_np2 = array.copy()
    camera_image_raw = pygame.surfarray.make_surface(array.swapaxes(0, 1))
    camera_image2 = pygame.transform.scale(camera_image_raw, (width, height))

camera.listen(process_image)
camera2.listen(process_image2)

# ========= Steering class -> steer value map =========
steering_map = {
    0: -0.8,   1: -0.6,   2: -0.4,   3: -0.25,  4: -0.20,  5: -0.10,  6: -0.05,
    7:  0.0,
    8:  0.05,  9:  0.10, 10:  0.15, 11:  0.25, 12:  0.40, 13:  0.60, 14:  0.80
}

# ========= PID settings (velocity control) =========
PROPORTIONAL_GAIN = 0.7
INTEGRAL_GAIN     = 0.02
DERIVATIVE_GAIN   = 0.28
INTEGRAL_CLAMP    = 5.0
MAX_ACCEL_THROTTLE = 1.0
MAX_BRAKE = 1.0
BRAKE_GAIN_MULT   = 2.2
MIN_BRAKE_TO_BITE = 0.15
IGNORE_ERROR      = 0.03

integral_err = 0.0
prev_err = 0.0
prev_time = time.time()

# ========= Hard-stop on Red Light =========
RED_CONF_THRESHOLD     = 0.70
RELEASE_CONF_THRESHOLD = 0.67
CONF_DROP_RELEASE      = 0.05   # >=7% absolute drop releases hard stop
hard_stop_active = False
prev_conf_tl = None            # track previous confidence for sudden-drop detection

# ========= Left-turn angle tracking =========
left_signal_on = False
left_turn_tracking_active = False
left_turn_deg = 0.0
left_turn_prev_yaw = None
prev_left_signal_on = False

def _wrap_deg(d: float) -> float:
    return (d + 180.0) % 360.0 - 180.0

def _delta_deg(prev: float, cur: float) -> float:
    return _wrap_deg(cur - prev)

# ========= Utility =========
def get_speed(actor: carla.Actor) -> float:
    v = actor.get_velocity()
    return float((v.x**2 + v.y**2 + v.z**2)**0.5)

# ========= Speed-aware steering attenuation =========
STEER_GAIN_LOW_V   = 3.0    # m/s  (~11 km/h)
STEER_GAIN_HIGH_V  = 20.0   # m/s  (~72 km/h)
STEER_GAIN_LOW     = 1.0
STEER_GAIN_HIGH    = 0.35   # reduce steering authority at higher speeds

def _lerp(a: float, b: float, t: float) -> float:
    t = max(0.0, min(1.0, t))
    return a + (b - a) * t

def _speed_gain(v: float) -> float:
    if v <= STEER_GAIN_LOW_V:
        return STEER_GAIN_LOW
    if v >= STEER_GAIN_HIGH_V:
        return STEER_GAIN_HIGH
    t = (v - STEER_GAIN_LOW_V) / (STEER_GAIN_HIGH_V - STEER_GAIN_LOW_V)
    return _lerp(STEER_GAIN_LOW, STEER_GAIN_HIGH, t)

clock = pygame.time.Clock()
print("Model-controlled driving started. Q/E/R signals, P respawn, ESC to exit.")

# ========= Respawn helper =========
def respawn_vehicle():
    global vehicle, camera, camera2
    try:
        if camera.is_listening:
            camera.stop()
        camera.destroy()
        if camera2.is_listening:
            camera2.stop()
        camera2.destroy()
    except Exception:
        pass
    try:
        vehicle.destroy()
    except Exception:
        pass

    sp = random.choice(world.get_map().get_spawn_points())
    vehicle_new = world.spawn_actor(vehicle_bp, sp)
    vehicle_new.set_autopilot(False)

    camera_new = world.spawn_actor(cam_bp, cam_transform, attach_to=vehicle_new)
    camera_new.listen(process_image)

    camera_new2 = world.spawn_actor(cam_bp2, cam_transform2, attach_to=vehicle_new)
    camera_new2.listen(process_image2)

    vehicle = vehicle_new
    camera = camera_new
    camera2 = camera_new2

# ========= Main loop =========
try:
    prev_steering_value = 0.0
    last_steering_time = time.time()
    STEERING_HOLD_S = 0.05

    while True:
        clock.tick(60)
        pygame.event.pump()
        keys = pygame.key.get_pressed()

        # Turn signals Q/E/R
        if keys[pygame.K_q]:
            vehicle.set_light_state(VehicleLightState.LeftBlinker)
            turn_signal = -1
        elif keys[pygame.K_e]:
            vehicle.set_light_state(VehicleLightState.RightBlinker)
            turn_signal = 1
        elif keys[pygame.K_r]:
            vehicle.set_light_state(VehicleLightState.NONE)
            turn_signal = 0
        else:
            ls = vehicle.get_light_state()
            if ls & VehicleLightState.LeftBlinker:
                turn_signal = -1
            elif ls & VehicleLightState.RightBlinker:
                turn_signal = 1
            else:
                turn_signal = 0

        if keys[pygame.K_p]:
            print("Respawning vehicle...")
            respawn_vehicle()
            time.sleep(0.5)
            continue

        # Track left/right signal state
        ls_now = vehicle.get_light_state()
        left_signal_on = bool(ls_now & VehicleLightState.LeftBlinker) or (turn_signal == -1)
        right_signal_on = bool(ls_now & VehicleLightState.RightBlinker) or (turn_signal == 1)

        # NEW: reset left-turn angle if NO signal is on
        if not left_signal_on and not right_signal_on:
            left_turn_deg = 0.0
            left_turn_tracking_active = False
            left_turn_prev_yaw = None

        # Existing logic: reset angle if right signal is on
        if right_signal_on:
            left_turn_deg = 0.0

        # Left-turn angle tracking start/stop
        if left_signal_on and not prev_left_signal_on:
            left_turn_tracking_active = True
            left_turn_deg = 0.0
            left_turn_prev_yaw = vehicle.get_transform().rotation.yaw
        elif (not left_signal_on) and prev_left_signal_on:
            left_turn_tracking_active = False
            left_turn_prev_yaw = None
        prev_left_signal_on = left_signal_on

        control = carla.VehicleControl()

        # ====== PREDICT (steering + velocity + traffic light) ======
        target_speed = None
        predicted_speed = None

        if camera_np is not None:
            with torch.no_grad():
                img_t = image_transform_steervel(camera_np).unsqueeze(0).to(device)

                # Steering
                signal_t = torch.tensor([[turn_signal + 1]], dtype=torch.long).to(device)
                out = steer_model(img_t, signal_t.squeeze(1))
                pred_class = out.argmax(dim=1).item()
                pred_label_steer = CLASS_NAMES_STEER[pred_class]
                steering_value = steering_map[pred_class]

                # Speed-aware attenuation
                current_speed_for_gain = get_speed(vehicle)  # m/s
                gain = _speed_gain(current_speed_for_gain)
                steering_value *= gain

                # Tiny minimum nudge when signal active
                if turn_signal == -1 and steering_value > -0.02:
                    steering_value = -0.02
                elif turn_signal == 1 and steering_value < 0.02:
                    steering_value = 0.02

                # Smooth/hold steering a touch
                now = time.time()
                if steering_value == 0.0 or prev_steering_value == 0.0:
                    control.steer = steering_value
                    prev_steering_value = steering_value
                    last_steering_time = now
                elif steering_value != prev_steering_value:
                    control.steer = (steering_value + prev_steering_value) / 2
                    prev_steering_value = steering_value
                    last_steering_time = now
                else:
                    if now - last_steering_time >= STEERING_HOLD_S:
                        control.steer = steering_value

                # Velocity regression -> target speed (m/s)
                v_out = velocity_model(img_t)
                raw_speed = float(v_out.squeeze().item())
                predicted_speed = max(0.0, raw_speed)
                target_speed = predicted_speed

        if camera_np2 is not None:
            with torch.no_grad():
                pred_label_tl, conf_tl = predict_traffic_light(camera_np2)

            name_is_red   = ("Red" in pred_label_tl) or ("🔴" in pred_label_tl)
            name_is_green = ("Green" in pred_label_tl) or ("🟢" in pred_label_tl)
            name_is_none  = ("No Light" in pred_label_tl) or ("⚫️" in pred_label_tl)

            # HARD STOP engage/disengage
            if TL_AVAILABLE:
                sudden_drop = (prev_conf_tl is not None) and ((prev_conf_tl - conf_tl) >= CONF_DROP_RELEASE)

                # Engage on confident RED
                if name_is_red and conf_tl >= RED_CONF_THRESHOLD:
                    hard_stop_active = True
                # Release if:
                #  - sudden confidence drop >= threshold (e.g., occlusion or misread clearing), OR
                #  - no longer red, OR
                #  - confidence fell below fixed floor
                elif hard_stop_active and (sudden_drop or (not name_is_red) or conf_tl < RELEASE_CONF_THRESHOLD):
                    hard_stop_active = False

            # update previous confidence after decision
            prev_conf_tl = conf_tl

        # ====== Throttle/Brake control ======
        throttle_cmd = 0.0
        brake_cmd = 0.0

        if hard_stop_active:
            throttle_cmd = 0.0
            brake_cmd = 1.0
            try:
                ls = vehicle.get_light_state()
                vehicle.set_light_state(ls | VehicleLightState.Brake)
            except Exception:
                pass
        else:
            if target_speed is not None:
                current_speed = get_speed(vehicle)
                speed_error = target_speed - current_speed

                now_t = time.time()
                dt = max(1e-3, now_t - prev_time)

                if abs(speed_error) < IGNORE_ERROR:
                    throttle_cmd = 0.0
                    brake_cmd = 0.0
                else:
                    # PID terms
                    integral_err = max(-INTEGRAL_CLAMP, min(INTEGRAL_CLAMP, integral_err + speed_error * dt))
                    rate_of_change = (speed_error - prev_err) / dt

                    pid_output = (
                        PROPORTIONAL_GAIN * speed_error
                        + INTEGRAL_GAIN * integral_err
                        + DERIVATIVE_GAIN * rate_of_change
                    )

                    if pid_output >= 0:
                        throttle_cmd = min(MAX_ACCEL_THROTTLE, max(0.0, pid_output))
                        brake_cmd = 0.0
                    else:
                        throttle_cmd = 0.0
                        brake_cmd = min(MAX_BRAKE, max(0.0, BRAKE_GAIN_MULT * (-pid_output)))
                        if 0.0 < brake_cmd < MIN_BRAKE_TO_BITE:
                            brake_cmd = MIN_BRAKE_TO_BITE
                        # reset integral when braking to avoid fighting the brakes
                        integral_err = 0.0

                prev_err = speed_error
                prev_time = now_t
            else:
                throttle_cmd = 0.0
                brake_cmd = 0.0

        control.throttle = throttle_cmd
        control.brake = brake_cmd
        control.hand_brake = False
        vehicle.apply_control(control)

        # Spectator follow
        tr = vehicle.get_transform()
        fwd = tr.get_forward_vector()
        cam_location = tr.location - fwd * 8 + carla.Location(z=3)
        cam_rotation = carla.Rotation(pitch=-10, yaw=tr.rotation.yaw)
        spectator.set_transform(carla.Transform(cam_location, cam_rotation))

        # Accumulate signed degrees during left turn
        if left_turn_tracking_active:
            cur_yaw = tr.rotation.yaw
            if left_turn_prev_yaw is not None:
                d = _delta_deg(left_turn_prev_yaw, cur_yaw)
                left_turn_deg += d
            left_turn_prev_yaw = cur_yaw

        # ====== HUD ======
        if camera_np2 is not None:
            frame_np = camera_np2
            if abs(left_turn_deg) > 2.5:
                g = cv2.cvtColor(frame_np, cv2.COLOR_RGB2GRAY)
                frame_np = cv2.cvtColor(g, cv2.COLOR_GRAY2RGB)

            camera_image2 = pygame.surfarray.make_surface(frame_np.swapaxes(0, 1))
            camera_image2 = pygame.transform.scale(camera_image2, (width, height))
            screen.blit(camera_image2, (0, 0))

            # Steering / speed text
            label_turn = font.render(f"Steer class: {pred_label_steer}", True, (255, 255, 0))
            screen.blit(label_turn, (20, 20))

            # Current steer value not stored globally; display from prev_steering_value (approx)
            label_steer = font.render(f"Steering (last): {prev_steering_value:.3f}", True, (0, 255, 0))
            screen.blit(label_steer, (20, 50))

            # Velocity HUD
            cur_speed = get_speed(vehicle)
            pred_speed_txt = f"Pred Speed: {predicted_speed:.2f} m/s" if predicted_speed is not None else "Pred Speed: -- m/s"
            label_pred_v = font.render(pred_speed_txt, True, (0, 255, 0))
            label_cur_v  = font.render(f"Current: {cur_speed:.2f} m/s", True, (0, 200, 255))
            screen.blit(label_pred_v, (20, 80))
            screen.blit(label_cur_v,  (20, 110))

            # Degrees since left indicator engaged
            label_turn_count = font.render(f"Left-turn Δ: {left_turn_deg:.1f}°", True, (255, 255, 255))
            screen.blit(label_turn_count, (20, 140))

            # Traffic-light banner
            banner = pygame.Surface((width, 46), pygame.SRCALPHA)
            banner.fill((0, 0, 0, 140))
            screen.blit(banner, (0, height - 46))
            tl_text = font_big.render(
                f"Traffic Light: {pred_label_tl} ({conf_tl*100:.1f}%)", True, (255, 255, 255)
            )
            screen.blit(tl_text, (20, height - 40))

            if hard_stop_active:
                tl_text2 = font_big.render("HARD STOP ACTIVE", True, (255, 80, 80))
                screen.blit(tl_text2, (width - 320, height - 40))

            pygame.display.flip()

        # Quit
        for event in pygame.event.get():
            if event.type == pygame.QUIT or keys[pygame.K_ESCAPE]:
                raise KeyboardInterrupt

except KeyboardInterrupt:
    print("Exiting and cleaning up...")
finally:
    try:
        camera.stop(); camera2.stop()
    except Exception:
        pass
    try:
        vehicle.destroy()
    except Exception:
        pass
    try:
        camera.destroy(); camera2.destroy()
    except Exception:
        pass
    pygame.quit()


Model-controlled driving started. Q/E/R signals, P respawn, ESC to exit.
Exiting and cleaning up...
